![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>


# PostgreSQL for Python Developers

## Hands-on PostgreSQL

In this project, you will practice using Python's DB-API to access PostgreSQL.

You will need access to a PostgreSQL installation where you have superuser permissions. If you do not have such access elsewhere, installing to your personal workstation is a good idea.  Alternately, you might wish to use a Docker container for a self-contained installation.  See `https://hub.docker.com/_/postgres` for details on that option.  Unless you have a specific need to work with an existing installation, choosing a PostgreSQL version of 12 or higher is best.


![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

## Part 1

**Move data into PostgreSQL**

In this respository, there is an SQLite database called `Airline-Tweets.sqlite`.  The lesson briefly mentioned `sqlite3` as a module in the standard library that will interact with single-file relational databases.  In particular, `sqlite3`, like `psycopg2` generally follows the DB-API.  Although more specifics of the PostgreSQL data definition language (DDL) is discussed in a later lesson, we assume you have a basic familiarity with the the SQL `CREATE TABLE` command for this purpose.

The scenario posted in this problem is that you have some data in a different database (SQLite), and you wish to transfer all contents to PostgreSQL.  Moreover, you wish to do this *entirely* within Python and using the DB-API.  You will need to determine the structure and data types of the SQLite data, create an appropriate PostgreSQL table, and transfer all rows into PostgreSQL.  This particular SQLite data file contains only one table, named `Tweets` (about customer ratings of airlines).

To get a partial view of the kind of data we are transferring, this is a query performed within the `sqlite` command shell, which is generally similar to the `psql` command shell that is discussed in later lessons.  However, you should perform all your work in Python for this project (the example does **not** list all the columns).

```
$ sqlite Airline-Tweets.sqlite
SQLite version 3.33.0 2020-08-14 13:23:32
Enter ".help" for usage hints.
sqlite> SELECT tweet_id, airline, airline_sentiment, tweet_created FROM Tweets LIMIT 5;
tweet_id            airline    airline_sentiment  tweet_created
------------------  ---------  -----------------  -------------------------
567588278875213824  Delta      neutral            2015-02-16 23:36:05 -0800
567590027375702016  Delta      negative           2015-02-16 23:43:02 -0800
567591480085463040  United     negative           2015-02-16 23:48:48 -0800
567592368451248130  United     negative           2015-02-16 23:52:20 -0800
567594449874587648  Southwest  negative           2015-02-17 00:00:36 -0800
```

In [ ]:
import psycopg2
import sqlite3
# ... your code here ...

**Hint**

Unfortunately, the `cursor.description` in `sqlite3` isn't as informative as one might like.  For guidance, you can look at this:

```
sqlite> pragma table_info('Tweets');
cid  name                          type     notnull  dflt_value  pk
---  ----------------------------  -------  -------  ----------  --
0    tweet_id                      INTEGER  0                    1
1    airline_sentiment             TEXT     0                    0
2    airline_sentiment_confidence  NUMERIC  0                    0
3    negativereason                TEXT     0                    0
4    negativereason_confidence     NUMERIC  0                    0
5    airline                       TEXT     0                    0
6    airline_sentiment_gold        TEXT     0                    0
7    name                          TEXT     0                    0
8    negativereason_gold           TEXT     0                    0
9    retweet_count                 INTEGER  0                    0
10   text                          TEXT     0                    0
11   tweet_coord                   TEXT     0                    0
12   tweet_created                 TEXT     0                    0
13   tweet_location                TEXT     0                    0
14   user_timezone                 TEXT     0                    0
```

It is wise also to examine some sample rows to decide the best PostgreSQL data types.  SQLite is more generic than PostgreSQL about data types.

**A possible solution**

You will need to make some decisions about data types and other details. These are plausible answers.  

In [ ]:
user, pwd, db = 'ine_student', 'ine-password', 'ine'
host, port = 'localhost', '5432'
con_dest = psycopg2.connect(database=db, host=host, user=user, password=pwd, port=port)
cur_dest = con_dest.cursor()

In [ ]:
# create PostgreSQL table
sql_create_tweets = '''
CREATE TABLE IF NOT EXISTS Tweets (
    tweet_id DECIMAL(18) PRIMARY KEY,
    airline_sentiment TEXT,
    airline_sentiment_confidence REAL,
    negativereason TEXT,
    negativereason_confidence REAL,
    airline TEXT,
    airline_sentiment_gold TEXT,
    name TEXT,
    negativereason_gold TEXT,
    retweet_count INT,
    text TEXT,
    tweet_coord TEXT,
    tweet_created TIMESTAMP WITH TIME ZONE,
    tweet_location TEXT,
    user_timezone TEXT
    );
'''
cur_dest.execute('DROP TABLE Tweets')
cur_dest.execute(sql_create_tweets)
con_dest.commit()

In [ ]:
con_src = sqlite3.connect('data/Airline-Tweets.sqlite') 
cur_src = con_src.cursor()
cur_src.execute("SELECT * FROM Tweets")

The obvious approach will fail because some numeric values are missing, but SQLite gives you an empty string rather than `None`.  We can fix that in Python.

In [5]:
sql_insert = """
INSERT INTO Tweets 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
for src_row in cur_src:
    row = tuple(data or None for data in src_row)
    cur_dest.execute(sql_insert, row)
    
con_dest.commit()

In [6]:
from pprint import pprint
cur_dest.execute("SELECT * FROM Tweets LIMIT 2;")
cols = [c.name for c in cur_dest.description]
for row in cur_dest:
    pprint(dict(zip(cols, row)))

{'airline': 'Delta',
 'airline_sentiment': 'neutral',
 'airline_sentiment_confidence': 1.0,
 'airline_sentiment_gold': None,
 'name': 'JetBlueNews',
 'negativereason': None,
 'negativereason_confidence': None,
 'negativereason_gold': None,
 'retweet_count': None,
 'text': "@JetBlue's new CEO seeks the right balance to please passengers and "
         'Wall ... - Greenfield Daily Reporter http://t.co/LM3opxkxch',
 'tweet_coord': None,
 'tweet_created': datetime.datetime(2015, 2, 17, 2, 36, 5, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-300, name=None)),
 'tweet_id': Decimal('567588278875213824'),
 'tweet_location': 'USA',
 'user_timezone': 'Sydney'}
{'airline': 'Delta',
 'airline_sentiment': 'negative',
 'airline_sentiment_confidence': 1.0,
 'airline_sentiment_gold': None,
 'name': 'nesi_1992',
 'negativereason': "Can't Tell",
 'negativereason_confidence': 0.6503,
 'negativereason_gold': None,
 'retweet_count': None,
 'text': '@JetBlue is REALLY getting on my nerves !! 😡😡 #nothappy',

![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)

## Part 2

**Skipping problem data when loading**

For this task, we will largely repeat the steps fo the first task.  However, you probably encountered certain problems with the data transfer that you had to manually remediate using Python code.  Rather than "fix up" the data in Python, for this version you should simply transfer the unproblematic data, but create another table called `data_issues` where you store the unique `tweet_id` of the problem row, and the error message that occurred.

In your solution, **some** data will transfer without remediation, and other data will be marked for later processing in this manner.

In [7]:
import psycopg2
import sqlite3
# ... your code here ...

**A possible solution**

We assume for this solution that the target table `Tweets` is created as in the prior part.  Also that the connections and cursors created in the efirst part are still live.

In [8]:
sql_create_issues = """
CREATE TABLE data_issues (
    tweet_id DECIMAL(18) PRIMARY KEY, 
    message TEXT
    );
"""
# Clear out old content of dest
cur_dest.execute('DROP TABLE data_issues;')
cur_dest.execute(sql_create_issues)
cur_dest.execute('DROP TABLE Tweets')
cur_dest.execute(sql_create_tweets)
con_dest.commit()

In [9]:
cur_src.execute("SELECT * FROM Tweets")

i = 0
for row in cur_src:
    try:
        cur_dest.execute(sql_insert, row)
    except Exception as err:
        con_dest.rollback()
        cur_dest.execute("INSERT INTO data_issues VALUES (%s, %s);",
                         (row[0], str(err)))
        con_dest.commit()
    else:
        con_dest.commit()

In [10]:
cur_dest.execute('SELECT count(*) FROM Tweets;')
cur_dest.fetchone()

[(10416,)]

In [11]:
cur_dest.execute("SELECT * FROM data_issues LIMIT 5;")
cur_dest.fetchall()

[(Decimal('567588278875213824'),
  'invalid input syntax for type real: ""\nLINE 3: VALUES (567588278875213824, \'neutral\', 1, \'\', \'\', \'Delta\', \'...\n                                                      ^\n'),
 (Decimal('567634106058821632'),
  'invalid input syntax for type real: ""\nLINE 3: VALUES (567634106058821632, \'neutral\', 1, \'\', \'\', \'United\', ...\n                                                      ^\n'),
 (Decimal('567643252753694721'),
  'invalid input syntax for type real: ""\nLINE 3: VALUES (567643252753694721, \'neutral\', 1, \'\', \'\', \'US Airway...\n                                                      ^\n'),
 (Decimal('567655489119326209'),
  'invalid input syntax for type real: ""\nLINE 3: VALUES (567655489119326209, \'positive\', 1, \'\', \'\', \'Southwes...\n                                                       ^\n'),
 (Decimal('567667301067915264'),
  'invalid input syntax for type real: ""\nLINE 3: VALUES (567667301067915264, \'neutral\', 1, 

In [12]:
cur_dest.execute("SELECT count(*) FROM data_issues;")
cur_dest.fetchone()

(4069,)

![orange-divider](https://user-images.githubusercontent.com/7065401/98619088-44ab6000-22e1-11eb-8f6d-5532e68ab274.png)
